In [1]:
# this notebook effectively does the same thing as "mzml_to_img.py" file, but in a notebook format
import sys
from decoder import *
import numpy as np
import time
from tqdm import tqdm
import cv2
# import matplotlib.pyplot as plt
import os
import math
file_name = os.listdir('src')
print(file_name)
s = input("index of file to process particular one: ")
if s != 'a':
    file_name = file_name[int(s)]

['.DS_Store', 'PXD010357-lungcancer-20.mzML', 'PXD010357-lungcancer-19.mzML']


In [2]:
file_path = 'src/'+file_name
t = time.time()
print(f'starting to process file {file_path}')

# decode the xml file and store the information in 'result', a list of 'spectrum' objects
listMz, listIn, listRt = decode(file_path)

# setting width and height, this can be any number we set

print('initializing min and max...')
min_mz = min([np.amin(e) for e in listMz])
max_mz = max([np.amax(e) for e in listMz])
min_rt = listRt[0]
max_rt = listRt[len(listRt)-1]

# adjust width and height here, bigger width and height yields higher resolution
# width and height must be an int
WIDTH = int(10*(max_mz - min_mz))
HEIGHT = int(len(listRt)/3)

starting to process file src/PXD010357-lungcancer-19.mzML
reading file...
parsing file...


100%|██████████| 41660/41660 [00:52<00:00, 796.64it/s] 


initializing min and max...


In [35]:
print('converting spectra to 2d img...')
# initialize img1 as a 2d array specified height and width
img = np.zeros((HEIGHT, WIDTH))
#listMz = [((e-min_mz)/(max_mz-min_mz)*(WIDTH-1)).astype('int') for e in tqdm(listMz)]
#listRt = ((np.array(listRt)-min_rt)/(max_rt-min_rt)*(HEIGHT-1)).astype('int')
for x_list, y, inten_list in zip(listMz, listRt, listIn):
    # print(img.shape, inten_list.shape)
    img[y, x_list] += inten_list

min_intensity = np.amin(img)
max_intensity = np.amax(img)
print('intensities:', max_intensity, min_intensity)

converting spectra to 2d img...
intensities: 1841302912.0 0.0


In [36]:
max_intensity/=500
img = np.clip(img, 0, max_intensity)

In [37]:
from scipy.special import logit
print('transforming intensities...')
img = (img - min_intensity) / (max_intensity - min_intensity) * 0.98 + 0.01
img = logit(img)
min_intensity = np.amin(img)
max_intensity = np.amax(img)
img = (img - min_intensity) / (max_intensity - min_intensity)
# img = np.tanh(5*img)
img *= 255
img = img.astype('int')

file_name = file_name.split('.')[0]
# LOOK HERE output is saved in the numpy array format in the misc folder and in image format in the img1 folder
np.save('misc/'+file_name+'_misc', np.array([max_rt, min_rt, max_mz, min_mz]))
print("image save status:" + str(cv2.imwrite('img1/' + file_name + ".png", img)))

transforming intensities...
image save status:True
